In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
df = pd.read_csv('./src/final_dataset.csv')
df.columns

Index(['date', 'season', 'league_name', 'match_result', 'team_long_name_home',
       'team_short_name_home', 'home_team_goal', 'home_players_average_rating',
       'home_players_average_score', 'home_team_score',
       'home_team_prob_receive_red_cards', 'home_team_prob_case_win',
       'team_long_name_away', 'team_short_name_away', 'away_team_goal',
       'away_players_average_rating', 'away_players_average_score',
       'away_team_score', 'away_team_prob_receive_red_cards',
       'away_team_prob_case_win', 'home_avg_bet', 'draw_avg_bet',
       'away_avg_bet'],
      dtype='object')

для начала подберем возможные варианты коэффициентов букмекеров для будущих игр

In [3]:
X = df[['home_players_average_rating', 'home_team_prob_case_win',
       'away_players_average_rating', 'away_team_prob_case_win']]
Y = df[['home_avg_bet', 'draw_avg_bet',
       'away_avg_bet']]

In [4]:
test_split_index = int(X.shape[0]*0.7)
X_train = X[:test_split_index]
X_test = X[test_split_index:]
Y_train = Y[:test_split_index]
Y_test = Y[test_split_index:]

In [5]:
model = LinearRegression()

In [6]:
model.fit(X_train, Y_train)

LinearRegression()

In [7]:
Y_predicted = model.predict(X_test)
bets_df=pd.DataFrame(Y_predicted, columns=['home_avg_bet1', 'draw_avg_bet1',
       'away_avg_bet1'])
bets_df

,home_avg_bet1,draw_avg_bet1,away_avg_bet1
0,0.855531,0.218229,0.002850
1,0.561889,0.267330,0.251686
2,0.557090,0.269571,0.250070
3,0.284239,0.302858,0.490594
4,0.702369,0.233008,0.138518
...,...,...,...
4470,0.656406,0.234543,0.169286
4471,0.541117,0.285236,0.237928
4472,0.706235,0.253850,0.103983
4473,0.427875,0.294041,0.342820


In [8]:
print('MSE: %.2f' % mean_squared_error(Y_test, Y_predicted))
print('MAE: %.2f' % mean_absolute_error(Y_test, Y_predicted))
print('R2: %.2f' % r2_score(Y_test, Y_predicted))

MSE: 0.00
MAE: 0.05
R2: 0.70


соберем все возможные комбинации команд внутри лиг

In [9]:
df1 = df[['league_name', 'team_long_name_home',
       'team_short_name_home',]]
df1.columns = ['league_name', 'team_long_name',
       'team_short_name']
df2 = df[['league_name', 'team_long_name_away',
       'team_short_name_away',]]
df2.columns = ['league_name', 'team_long_name',
       'team_short_name']
concat_df = pd.concat([df1, df2], ignore_index=True)
concat_df.drop_duplicates(inplace=True)
cross_joined_df = pd.concat(
    [
        sub_df.merge(sub_df, on='league_name').query('team_long_name_x != team_long_name_y')
        for _, sub_df in concat_df.groupby('league_name')
    ]
)
cross_joined_df.columns = ['league_name', 'team_long_name_home',
       'team_short_name_home','team_long_name_away',
       'team_short_name_away']
cross_joined_df

,league_name,team_long_name_home,team_short_name_home,team_long_name_away,team_short_name_away
1,Belgium Jupiler League,RSC Anderlecht,AND,KV Mechelen,MEC
2,Belgium Jupiler League,RSC Anderlecht,AND,Standard de Liège,STL
3,Belgium Jupiler League,RSC Anderlecht,AND,Beerschot AC,BAC
4,Belgium Jupiler League,RSC Anderlecht,AND,KRC Genk,GEN
5,Belgium Jupiler League,RSC Anderlecht,AND,SV Zulte-Waregem,ZUL
...,...,...,...,...,...
955,Spain LIGA BBVA,UD Las Palmas,LAS,Granada CF,GRA
956,Spain LIGA BBVA,UD Las Palmas,LAS,RC Celta de Vigo,CEL
957,Spain LIGA BBVA,UD Las Palmas,LAS,Elche CF,ELC
958,Spain LIGA BBVA,UD Las Palmas,LAS,SD Eibar,EIB


подтянем известные данные, которые впоследствии будут использованы как независимые переменные

In [10]:
temp_df=df
temp_df['date'] = pd.to_datetime(temp_df['date'])
temp_df=temp_df.sort_values('date').drop_duplicates(['league_name', 'team_long_name_home',
       'team_short_name_home', 'team_short_name_away', 'team_long_name_away',], keep='last')
cross_joined_df=pd.merge(cross_joined_df, temp_df, on=['league_name', 'team_long_name_home',
       'team_short_name_home','team_long_name_away',
       'team_short_name_away'], how='left')

In [11]:
cross_joined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6532 entries, 0 to 6531
Data columns (total 23 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   league_name                       6532 non-null   object        
 1   team_long_name_home               6532 non-null   object        
 2   team_short_name_home              6532 non-null   object        
 3   team_long_name_away               6532 non-null   object        
 4   team_short_name_away              6532 non-null   object        
 5   date                              4975 non-null   datetime64[ns]
 6   season                            4975 non-null   object        
 7   match_result                      4975 non-null   float64       
 8   home_team_goal                    4975 non-null   float64       
 9   home_players_average_rating       4975 non-null   float64       
 10  home_players_average_score        4975 non-null 

заполнились только те строчки, в которых home team и away team уже играли друг с другом на тех же полях. для остальных строк данные нужно подтянуть 2 путями: отдельно для переменных, которые не зависят от того, играет ли команда на своем поле или нет. начнем с players_average_rating и team_score, значения которых не привязаны к параметру home/away.

In [12]:
temp_df

,date,season,league_name,match_result,team_long_name_home,team_short_name_home,home_team_goal,home_players_average_rating,home_players_average_score,home_team_score,...,team_short_name_away,away_team_goal,away_players_average_rating,away_players_average_score,away_team_score,away_team_prob_receive_red_cards,away_team_prob_case_win,home_avg_bet,draw_avg_bet,away_avg_bet
12778,2010-01-03,2009/2010,Spain LIGA BBVA,1,Racing Santander,SAN,2,74.00,67.20,56.88,...,TEN,0,72.00,64.91,40.62,0.000000,0.000000,0.505051,0.303030,0.277008
12780,2010-01-03,2009/2010,Spain LIGA BBVA,1,UD Almería,ALM,1,74.00,66.79,54.38,...,XER,0,70.27,64.15,43.75,0.000000,33.333333,0.574713,0.292398,0.214133
10741,2010-01-10,2009/2010,Portugal Liga ZON Sagres,1,Académica de Coimbra,ACA,2,67.18,63.13,39.38,...,NAV,0,66.36,61.47,39.38,0.000000,28.571429,0.515464,0.324675,0.265252
12787,2010-01-10,2009/2010,Spain LIGA BBVA,-1,CD Tenerife,TEN,0,72.00,64.91,40.62,...,BAR,5,83.73,76.37,52.12,0.000000,66.964286,0.135501,0.226757,0.724638
3114,2010-01-13,2009/2010,France Ligue 1,0,US Boulogne Cote D'Opale,BOU,0,64.82,59.37,53.12,...,SOC,0,68.73,62.30,52.62,0.000000,16.279070,0.334448,0.333333,0.418410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12743,2015-12-30,2015/2016,Scotland Premier League,1,Motherwell,MOT,2,60.64,55.11,45.88,...,JOH,0,63.73,57.66,48.25,0.000000,32.352941,0.353357,0.303030,0.409836
3028,2015-12-30,2015/2016,England Premier League,-1,Sunderland,SUN,0,74.82,66.27,49.88,...,LIV,1,78.64,69.09,48.62,3.539823,38.053097,0.198413,0.271739,0.581395
14837,2015-12-30,2015/2016,Spain LIGA BBVA,0,Getafe CF,GET,0,74.00,66.86,43.62,...,COR,0,73.55,64.74,49.88,4.054054,12.162162,0.392157,0.315457,0.346021
14843,2015-12-30,2015/2016,Spain LIGA BBVA,-1,Rayo Vallecano,RAY,0,73.60,63.48,47.12,...,AMA,2,79.18,68.72,52.75,6.363636,43.636364,0.158228,0.252525,0.636943


In [13]:
# заполним рейтинги и очки команд для новых комбинаций пар
temp_df1 = temp_df[['date', 'team_long_name_home',
       'team_short_name_home','home_players_average_rating',
       'home_team_score']]
temp_df1.columns = ['date', 'team_long_name',
       'team_short_name','players_average_rating',
       'team_score']
temp_df2 = temp_df[['date', 'team_long_name_away',
       'team_short_name_away','away_players_average_rating',
       'away_team_score']]
temp_df2.columns = ['date', 'team_long_name',
       'team_short_name','players_average_rating',
       'team_score']
concat_temp_df = pd.concat([temp_df1, temp_df2], ignore_index=True)
concat_temp_df=concat_temp_df.sort_values('date').drop_duplicates(['team_long_name', 'team_short_name'], keep='last')
cross_joined_df['home_players_average_rating']=cross_joined_df['home_players_average_rating'].fillna(cross_joined_df.apply(lambda x: concat_temp_df[concat_temp_df['team_long_name'] == x['team_long_name_home'] ]['players_average_rating'].values[0] if pd.isnull(x['home_players_average_rating']) else x['home_players_average_rating'], axis=1))
cross_joined_df['away_players_average_rating']=cross_joined_df['away_players_average_rating'].fillna(cross_joined_df.apply(lambda x: concat_temp_df[concat_temp_df['team_long_name'] == x['team_long_name_home'] ]['players_average_rating'].values[0] if pd.isnull(x['away_players_average_rating']) else x['away_players_average_rating'], axis=1))
cross_joined_df['home_team_score']=cross_joined_df['home_team_score'].fillna(cross_joined_df.apply(lambda x: concat_temp_df[concat_temp_df['team_long_name'] == x['team_long_name_home'] ]['team_score'].values[0] if pd.isnull(x['home_team_score']) else x['home_team_score'], axis=1))
cross_joined_df['away_team_score']=cross_joined_df['away_team_score'].fillna(cross_joined_df.apply(lambda x: concat_temp_df[concat_temp_df['team_long_name'] == x['team_long_name_home'] ]['team_score'].values[0] if pd.isnull(x['away_team_score']) else x['away_team_score'], axis=1))

далее подберем параметры, значения которых зависят от поля (home/away). заполним team_prob_receive_red_cards и team_prob_case_win

In [14]:
temp_df3 = temp_df[['date', 'team_long_name_home',
       'team_short_name_home','home_team_prob_receive_red_cards', 'home_team_prob_case_win']]
temp_df3=temp_df3.sort_values('date').drop_duplicates(['team_long_name_home', 'team_short_name_home'], keep='last')
temp_df3['date'] = temp_df3['date'].dt.year
temp_df4 = temp_df[['date', 'team_long_name_away',
       'team_short_name_away','away_team_prob_receive_red_cards', 'away_team_prob_case_win']]
temp_df4=temp_df4.sort_values('date').drop_duplicates(['team_long_name_away', 'team_short_name_away'], keep='last')
temp_df4['date'] = temp_df4['date'].dt.year
temp_df3

,date,team_long_name_home,team_short_name_home,home_team_prob_receive_red_cards,home_team_prob_case_win
12837,2010,CD Tenerife,TEN,0.000000,0.000000
9326,2010,Sparta Rotterdam,SPA,0.000000,14.285714
10790,2010,Leixões SC,LEI,0.000000,0.000000
1043,2010,Portsmouth,POR,11.111111,22.222222
11771,2010,Falkirk,FAL,0.000000,11.111111
...,...,...,...,...,...
14841,2015,RC Celta de Vigo,CEL,6.349206,36.507937
14842,2015,Levante UD,LEV,9.183673,41.836735
14843,2015,Rayo Vallecano,RAY,10.389610,42.857143
12741,2015,Aberdeen,ABE,0.000000,43.750000


одна из команд никогда не играла на своем поле. нужно найти ее и присвоить значение, исходя из аналогичного параметра для away

In [15]:
missed_team=temp_df4[ ~ temp_df4['team_long_name_away']. isin (temp_df3['team_long_name_home'])]
missed_team

,date,team_long_name_away,team_short_name_away,away_team_prob_receive_red_cards,away_team_prob_case_win
16,2010,KSV Roeselare,ROS,0.0,50.0


рассчитаем, насколько home_team_prob_receive_red_cards и home_team_prob_case_win больше/меньше away_team_prob_receive_red_cards и away_team_prob_case_win в среднем. 

In [16]:
temp_df3.columns= ['date', 'team_long_name_away',
       'team_short_name_away','home_team_prob_receive_red_cards', 'home_team_prob_case_win']
temp_df3

,date,team_long_name_away,team_short_name_away,home_team_prob_receive_red_cards,home_team_prob_case_win
12837,2010,CD Tenerife,TEN,0.000000,0.000000
9326,2010,Sparta Rotterdam,SPA,0.000000,14.285714
10790,2010,Leixões SC,LEI,0.000000,0.000000
1043,2010,Portsmouth,POR,11.111111,22.222222
11771,2010,Falkirk,FAL,0.000000,11.111111
...,...,...,...,...,...
14841,2015,RC Celta de Vigo,CEL,6.349206,36.507937
14842,2015,Levante UD,LEV,9.183673,41.836735
14843,2015,Rayo Vallecano,RAY,10.389610,42.857143
12741,2015,Aberdeen,ABE,0.000000,43.750000


In [17]:
temp_df3 = pd.merge(temp_df3, temp_df4, on=['date', 'team_long_name_away',
       'team_short_name_away'], how='right')
temp_df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 7 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   date                              242 non-null    int32  
 1   team_long_name_away               242 non-null    object 
 2   team_short_name_away              242 non-null    object 
 3   home_team_prob_receive_red_cards  241 non-null    float64
 4   home_team_prob_case_win           241 non-null    float64
 5   away_team_prob_receive_red_cards  242 non-null    float64
 6   away_team_prob_case_win           242 non-null    float64
dtypes: float64(4), int32(1), object(2)
memory usage: 12.4+ KB


In [18]:
check_joined_df = temp_df3[(temp_df3.away_team_prob_receive_red_cards != 0 ) & (temp_df3.away_team_prob_case_win != 0 )]
check_joined_df.dropna ()
check_joined_df['prop_red_cards']=check_joined_df['home_team_prob_receive_red_cards']/check_joined_df['away_team_prob_receive_red_cards']
check_joined_df['prop_prob_case_win']=check_joined_df['home_team_prob_case_win']/check_joined_df['away_team_prob_case_win']
print(check_joined_df['prop_red_cards'].mean(skipna=True))
print(check_joined_df['prop_prob_case_win'].mean(skipna=True))

0.8876967909306577
1.7588047510271134


C:\Users\123\AppData\Local\Temp\ipykernel_11716\2267830606.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check_joined_df['prop_red_cards']=check_joined_df['home_team_prob_receive_red_cards']/check_joined_df['away_team_prob_receive_red_cards']
C:\Users\123\AppData\Local\Temp\ipykernel_11716\2267830606.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check_joined_df['prop_prob_case_win']=check_joined_df['home_team_prob_case_win']/check_joined_df['away_team_prob_case_win']


заполним значения home_team_prob_receive_red_cards и home_team_prob_case_win для команды KSV Roeselare, которая никогда не играла на своем поле

In [19]:
temp_df3.columns= ['date', 'team_long_name',
       'team_short_name','home_team_prob_receive_red_cards', 'home_team_prob_case_win','away_team_prob_receive_red_cards', 'away_team_prob_case_win']
temp_df3['home_team_prob_receive_red_cards']=temp_df3['home_team_prob_receive_red_cards'].fillna(temp_df3['away_team_prob_receive_red_cards']*check_joined_df['prop_red_cards'].mean(skipna=True))
temp_df3['home_team_prob_case_win']=temp_df3['home_team_prob_case_win'].fillna(temp_df3['away_team_prob_case_win']*check_joined_df['prop_prob_case_win'].mean(skipna=True))
temp_df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 7 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   date                              242 non-null    int32  
 1   team_long_name                    242 non-null    object 
 2   team_short_name                   242 non-null    object 
 3   home_team_prob_receive_red_cards  242 non-null    float64
 4   home_team_prob_case_win           242 non-null    float64
 5   away_team_prob_receive_red_cards  242 non-null    float64
 6   away_team_prob_case_win           242 non-null    float64
dtypes: float64(4), int32(1), object(2)
memory usage: 12.4+ KB


In [20]:
cross_joined_df['home_team_prob_receive_red_cards']=cross_joined_df['home_team_prob_receive_red_cards'].fillna(cross_joined_df.apply(lambda x: temp_df3[temp_df3['team_long_name'] == x['team_long_name_home'] ]['home_team_prob_receive_red_cards'].values[0] if pd.isnull(x['home_team_prob_receive_red_cards']) else x['home_team_prob_receive_red_cards'], axis=1))
cross_joined_df['away_team_prob_receive_red_cards']=cross_joined_df['away_team_prob_receive_red_cards'].fillna(cross_joined_df.apply(lambda x: temp_df3[temp_df3['team_long_name'] == x['team_long_name_away'] ]['away_team_prob_receive_red_cards'].values[0] if pd.isnull(x['away_team_prob_receive_red_cards']) else x['away_team_prob_receive_red_cards'], axis=1))
cross_joined_df['home_team_prob_case_win']=cross_joined_df['home_team_prob_case_win'].fillna(cross_joined_df.apply(lambda x: temp_df3[temp_df3['team_long_name'] == x['team_long_name_home'] ]['home_team_prob_case_win'].values[0] if pd.isnull(x['home_team_prob_case_win']) else x['home_team_prob_case_win'], axis=1))
cross_joined_df['away_team_prob_case_win']=cross_joined_df['away_team_prob_case_win'].fillna(cross_joined_df.apply(lambda x: temp_df3[temp_df3['team_long_name'] == x['team_long_name_away'] ]['away_team_prob_case_win'].values[0] if pd.isnull(x['away_team_prob_case_win']) else x['away_team_prob_case_win'], axis=1))

In [21]:
predicted_bets=model.predict(cross_joined_df[['home_players_average_rating',
       'home_team_prob_case_win', 'away_players_average_rating',
       'away_team_prob_case_win']])
cross_joined_df[['home_avg_bet', 'draw_avg_bet',
       'away_avg_bet']]=predicted_bets

In [22]:
cross_joined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6532 entries, 0 to 6531
Data columns (total 23 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   league_name                       6532 non-null   object        
 1   team_long_name_home               6532 non-null   object        
 2   team_short_name_home              6532 non-null   object        
 3   team_long_name_away               6532 non-null   object        
 4   team_short_name_away              6532 non-null   object        
 5   date                              4975 non-null   datetime64[ns]
 6   season                            4975 non-null   object        
 7   match_result                      4975 non-null   float64       
 8   home_team_goal                    4975 non-null   float64       
 9   home_players_average_rating       6532 non-null   float64       
 10  home_players_average_score        4975 non-null 

In [28]:
cross_joined_df.drop(['date', 'season', 'match_result', 'home_team_goal', 'home_players_average_score', 'away_team_goal', 'away_players_average_score'], axis= 1 , inplace= True )

In [30]:
cross_joined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6532 entries, 0 to 6531
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   league_name                       6532 non-null   object 
 1   team_long_name_home               6532 non-null   object 
 2   team_short_name_home              6532 non-null   object 
 3   team_long_name_away               6532 non-null   object 
 4   team_short_name_away              6532 non-null   object 
 5   home_players_average_rating       6532 non-null   float64
 6   home_team_score                   6532 non-null   float64
 7   home_team_prob_receive_red_cards  6532 non-null   float64
 8   home_team_prob_case_win           6532 non-null   float64
 9   away_players_average_rating       6532 non-null   float64
 10  away_team_score                   6532 non-null   float64
 11  away_team_prob_receive_red_cards  6532 non-null   float64
 12  away_t

In [31]:
cross_joined_df.to_csv(
    './src/new_variables.csv', index=False)